In [1]:
import ast

import pandas as pd
from Bio import Entrez

from Scripts.fasta_processing import dict_align_to_fasta
from data_processing import analyze_exons
from parse_psi_blast_results import parse_psiblast_xml, filter_psiblast_hits, update_df, save_files, create_many_cassettes, dict_align_info_analyze

Entrez.email = "artemvaskaa@gmail.com"

In [2]:
# name_of_blast_res = "../Blast_res/schistosoma_haematobium_psi_blast.xml"  # XML2 !!!
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
# df.to_csv("../Sequences_protein_id/df_no_filters.tsv", sep="\t", index=True)

In [3]:
# save_files(df)

---

In [4]:
df = pd.read_csv("../Sequences_protein_id/df_no_filters.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

prefix = "../Sequences_protein_id"
ref_exon_len = [37]
found_protein_ids = {}

for protein_id in df.protein_id:
    df_subset = df[df["protein_id"] == protein_id]
    df_exons = analyze_exons(f"{prefix}/{protein_id}/exons.fa")
    condition = set(ref_exon_len) & set(df_exons.length.tolist())
    if condition:
        found_protein_ids[protein_id] = condition

df_found_protein_ids = df[df["protein_id"].isin(found_protein_ids.keys())]

data = {}

for protein_id in df_found_protein_ids.protein_id:
    df_subset = df_found_protein_ids[df_found_protein_ids["protein_id"] == protein_id]
    org_name = df_subset.org_name.iloc[0]
    df_exons = analyze_exons(f"{prefix}/{protein_id}/exons.fa")
    exon_37_idx = df_exons[df_exons["length"] == 37].index[0]
    exon_110_idx = exon_37_idx - 1
    data[f"{org_name}_{protein_id}"] = (df_exons, [exon_110_idx, exon_37_idx])

In [5]:
introns = create_many_cassettes("../Sequences_protein_id", data)

In [6]:
df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, "cds")
df_cds.equal_to_cds.unique()  # должно быть только True

array([ True])

In [7]:
df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(df_found_protein_ids, "cds_cassette")
df_cds_cassette

,org_name_protein_id,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length
0,Schistosoma_guineensis_CAH8656942.1,981,False,980,1,652
1,Schistosoma_curassoni_CAH8663826.1,981,False,980,1,652
2,Schistosoma_bovis_CAH8671530.1,981,False,980,1,652
3,Schistosoma_margrebowiei_CAH8667553.1,981,False,980,1,650
4,Schistosoma_intercalatum_CAH8646207.1,981,False,980,1,652
5,Schistosoma_rodhaini_CAH8677847.1,981,False,980,1,671
6,Schistosoma_japonicum_TNN16962.1,978,False,977,1,847
7,Schistosoma_turkestanicum_CAH8587689.1,978,False,977,1,905
8,Paragonimus_westermani_KAF8560478.1,993,False,992,1,13971
9,Taenia_crassiceps_KAL5103217.1,981,False,980,1,278


---

In [8]:
# dict_align_to_fasta(dict_align_cds_cassette, "../Alignment/psi_blast/cds_cassette.fa")
# dict_align_to_fasta(dict_align_cds_cassette, "../Alignment/psi_blast/cds_cassette.aln")